In [3]:
import sys
sys.path.append("..")

from citibikeAPI import *
from utils import *


In [4]:
d = getAllStationData()
# trip_data = pd.read_csv('../../raw/202003-citibike-tripdata.csv')
# adjacenyDict = findMostCommonRoutesByStation(trip_data)


In [5]:
def findMostCommonRoutesByStation(trip_data):
	station_data = getAllStationData()
	#we only want trips that dont start and end at the same station
	trips = trip_data[trip_data['start station id'] != trip_data['end station id']].value_counts(['start station name','end station name'])
	#find unique starting stations
	unique_start_stations = list(set([trips.index[i][0] for i in range(len(trips))]))
	adjacencyDict = {}
	for station in unique_start_stations: 
		try: 
			id = station_data.index[station_data['name']==station][0]
			top_ten_routes = trips[station][:10]
			edges = []
			for route in top_ten_routes.index: 
				edges.append(station_data.index[station_data['name']==route][0])
			adjacencyDict[id] = edges
		#sometimes we get an index error if a station went online / offline since the last data pull 
		except IndexError: 
			pass 
	return adjacencyDict

In [6]:
d.head()
locData = d
locData = locData.reset_index()
locData.head()

,station_id,name,lat,lon,capacity,legacy_id,num_docks_available,num_bikes_disabled,num_ebikes_available,num_bikes_available,num_docks_disabled,station_status,is_renting,is_returning,last_reported,is_installed
0,c3189056-8eda-4278-891f-e8e15c2d759a,Carlton Ave & Dean St,40.680974,-73.971010,29,3537,0,0,0,0,0,out_of_service,0,0,1678109761,0
1,3e1bdcec-e762-4e83-b447-007f05923cce,19 St & 24 Ave,40.777455,-73.922540,23,3524,14,1,1,8,0,active,1,1,1680615371,1
2,517409d3-a94a-4140-a6de-ff0de9221ed2,Westchester Ave & E 156 St,40.817802,-73.905203,23,3983,20,1,1,1,0,active,1,1,1680615371,1
3,66dce9b5-0aca-11e7-82f6-3863bb44ef7c,Division Ave & Marcy Ave,40.707087,-73.957968,33,3075,12,1,1,20,0,active,1,1,1680615375,1
4,66dbcbbc-0aca-11e7-82f6-3863bb44ef7c,West Thames St,40.708347,-74.017134,73,363,18,1,2,54,0,active,1,1,1680615371,1


In [9]:
#min max lat/long for normalize
# need to filter out zeros for some data noise 
minlat = min([i for i in d['lat'] if i > 0])
maxlat  = max([i for i in d['lat'] if i > 0])
minlong = min([i for i in d['lon'] if i  < 0])
maxlong = max([i for i in d['lon'] if i < 0])

#add two columns and normalize
#hardcoding column locations to overwrite ""
d['x'] = 0
d['y'] = 0
# d['adjacency'] = ''
locData = d
locData = locData.reset_index()
# print(adjacenyDict)
for r in range(d.shape[0]):
    d.iloc[r,-2] = clamp((d.iloc[r, 1] - minlat)/(maxlat-minlat),0,1) #lat
    d.iloc[r,-1] = clamp((d.iloc[r, 2] - minlong)/(maxlong-minlong),0,1) #lat
    # try: 
    #     d.iloc[r, -1] = ' '.join([str(locData.index[locData['station_id']==v][0]) for v in adjacenyDict[locData.iloc[r,0]]])
    # except KeyError: pass

locData = d[['x', 'y']]


In [10]:
locData.head()

,x,y
station_id,,
c3189056-8eda-4278-891f-e8e15c2d759a,0,0.191217
3e1bdcec-e762-4e83-b447-007f05923cce,0,0.578886
517409d3-a94a-4140-a6de-ff0de9221ed2,0,0.741003
66dce9b5-0aca-11e7-82f6-3863bb44ef7c,0,0.296141
66dbcbbc-0aca-11e7-82f6-3863bb44ef7c,0,0.301203


In [ ]:
locData.to_csv('../../clean/locdata.csv')